In [ ]:
# Korean Text Categorization Performance Test on Google Colab
## MPS vs CUDA vs CPU 성능 비교

이 노트북은 한국어 텍스트 분류 성능을 다양한 하드웨어에서 테스트합니다.

**주요 기능:**
- V1 (단순 카테고리 이름 비교) vs V2 (NER + 키워드 평균 임베딩) 비교
- CPU, CUDA, MPS 성능 벤치마크
- 실제 한국어 일정 데이터를 이용한 테스트


In [ ]:
# 필요한 패키지 설치
!pip install sentence-transformers transformers torch scikit-learn pandas numpy tqdm matplotlib


In [ ]:
import torch
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import re
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt


In [ ]:
# 하드웨어 정보 확인
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"CUDA device count: {torch.cuda.device_count()}")

# MPS 지원 확인 (Apple Silicon)
print(f"MPS available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False}")

# 사용할 디바이스 결정
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f"Using device: {device}")


In [ ]:
# 테스트 데이터 (실제 프로젝트의 일부 데이터)
test_data = [
    ("깃헙 회의", ["학업", "친목", "업무"]),
    ("자료구조 과제 제출", ["학업"]),
    ("알고리즘 스터디 모임", ["학업", "친목"]),
    ("백준 풀기", ["학업", "취미"]),
    ("운영체제 중간고사 공부 계획 세우기", ["학업", "시험"]),
    ("졸업 프로젝트 아이디어 구상 및 주제 선정", ["학업"]),
    ("국가근로장학금 신청 서류 제출", ["경제", "재무", "학업"]),
    ("노트북 쿨링팬 청소 및 서멀구리스 재도포", ["취미", "가사"]),
    ("월세 및 통신비 이체", ["정기 지출", "경제"]),
    ("헌혈의 집 방문하여 헌혈하기", ["봉사", "건강"]),
    ("라식 수술 상담 예약", ["건강", "예약", "미용"]),
    ("친구들과 보드게임 카페 방문", ["친목", "취미"]),
    ("중고 전공서적 판매글 올리기", ["구매", "경제"]),
    ("시댁 방문 및 김장 도와드리기", ["가족", "봉사"]),
    ("반려견 산책 및 동물병원 방문", ["반려 동물", "건강"]),
    ("과제 채점하기", ["업무", "학업", "시험"]),
    ("발표대본 만들기", ["업무", "학업", "시험"]),
    ("브랜드 협찬 제품 리뷰 영상 기획", ["업무", "구매"]),
    ("애인과 커플 요금제 상담", ["연애", "통화", "경제"]),
    ("부모님 스마트폰 백업 도와드리기", ["가족", "봉사"])
]

print(f"Test data loaded: {len(test_data)} samples")


In [ ]:
# 카테고리 정의 (개선된 키워드 버전) - 주요 카테고리만
CATEGORIES = {
    "학업": [
        "과제 작성하고 제출하기", "시험 준비하고 공부하기", "수업 듣고 참여하기", 
        "논문 작성하고 연구하기", "프로젝트 기획하고 진행하기", "스터디 참여하고 토론하기",
        "학점 관리하고 성적 확인하기", "졸업 요건 확인하고 준비하기", "전공 심화 학습하기",
        "학술 활동 참여하기", "도서관에서 자료 조사하기", "온라인 강의 수강하기"
    ],
    "업무": [
        "회의 참석하고 업무 논의하기", "업무 계획 수립하고 실행하기", "프레젠테이션 준비하고 발표하기",
        "클라이언트와 미팅하고 상담하기", "업무 보고서 작성하고 제출하기", "프로젝트 관리하고 진행하기",
        "동료와 협업하고 소통하기", "업무 스킬 향상하고 교육받기", "성과 평가받고 피드백하기",
        "업무 환경 개선하고 최적화하기", "고객 서비스 제공하고 응대하기", "팀 업무 조율하고 관리하기"
    ],
    "건강": [
        "운동하고 체력 관리하기", "병원 방문하고 검진받기", "약 복용하고 치료받기",
        "건강한 식단 관리하고 섭취하기", "정기 건강검진 받기", "스트레스 관리하고 휴식하기",
        "충분한 수면 취하고 컨디션 관리하기", "금연 금주하고 건강 습관 유지하기",
        "정신 건강 관리하고 상담받기", "예방접종 받고 감염 예방하기"
    ],
    "경제": [
        "가계부 작성하고 지출 관리하기", "적금 저축하고 재정 계획하기", "투자 상품 알아보고 운용하기",
        "용돈 관리하고 예산 세우기", "할인 혜택 찾고 절약하기", "부채 관리하고 상환하기",
        "가격 비교하고 합리적 소비하기", "수입 늘리고 부업 알아보기", "보험료 납부하고 보장받기"
    ],
    "친목": [
        "친구들과 만나서 수다떨기", "동료들과 회식하고 친해지기", "모임 참석하고 네트워킹하기",
        "지인들과 취미 활동 함께하기", "새로운 사람들과 인맥 쌓기", "가족 모임 참석하고 소통하기",
        "커뮤니티 활동 참여하고 교류하기", "파티 참석하고 즐기기"
    ],
    "취미": [
        "독서하고 책 감상하기", "영화 보고 드라마 시청하기", "음악 듣고 악기 연주하기",
        "게임하고 오락 즐기기", "요리하고 베이킹하기", "운동하고 스포츠 활동하기",
        "여행 계획하고 떠나기", "사진 촬영하고 편집하기", "공예 만들고 DIY하기"
    ]
}

print(f"Categories defined: {len(CATEGORIES)} categories")
for cat, keywords in CATEGORIES.items():
    print(f"{cat}: {len(keywords)} keywords")


In [ ]:
class PerformanceTester:
    def __init__(self, device='cpu'):
        self.device = device
        print(f"Initializing models on {device}...")
        
        # SentenceTransformer 모델 로드
        self.sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
        if device != 'cpu':
            self.sentence_model.to(device)
        
        # NER 모델 로드
        self.ner_pipeline = pipeline(
            "ner",
            model="Leo97/KoELECTRA-small-v3-modu-ner",
            tokenizer="Leo97/KoELECTRA-small-v3-modu-ner",
            aggregation_strategy="simple",
            device=0 if device == 'cuda' else -1
        )
        
        # 카테고리 임베딩 미리 계산
        self._precompute_category_embeddings()
        
    def _precompute_category_embeddings(self):
        """카테고리 임베딩을 미리 계산"""
        print("Precomputing category embeddings...")
        
        # V1: 단순 카테고리 이름
        self.category_embeddings_v1 = {}
        category_names = list(CATEGORIES.keys())
        embeddings = self.sentence_model.encode(category_names)
        for name, embedding in zip(category_names, embeddings):
            self.category_embeddings_v1[name] = embedding
            
        # V2: 키워드 평균 임베딩
        self.category_embeddings_v2 = {}
        for category, keywords in CATEGORIES.items():
            keyword_embeddings = self.sentence_model.encode(keywords)
            avg_embedding = np.mean(keyword_embeddings, axis=0)
            self.category_embeddings_v2[category] = avg_embedding
            
    def preprocess_with_ner(self, text: str) -> str:
        """NER로 텍스트 전처리"""
        try:
            entities = self.ner_pipeline(text)
            processed_text = text
            
            # 엔티티를 플레이스홀더로 변경 (긴 것부터 처리)
            entities = sorted(entities, key=lambda x: x['start'], reverse=True)
            
            for entity in entities:
                if entity['entity_group'] == 'PER':
                    placeholder = '<PERSON>'
                elif entity['entity_group'] == 'LOC':
                    placeholder = '<LOCATION>'
                elif entity['entity_group'] == 'ORG':
                    placeholder = '<ORG>'
                else:
                    continue
                    
                processed_text = (
                    processed_text[:entity['start']] + 
                    placeholder + 
                    processed_text[entity['end']:]
                )
                
            return processed_text
        except:
            return text
            
    def predict_v1(self, text: str, threshold: float = 0.5) -> List[str]:
        """V1: 단순 카테고리 이름 비교"""
        text_embedding = self.sentence_model.encode([text])[0]
        
        predictions = []
        for category, category_embedding in self.category_embeddings_v1.items():
            similarity = cosine_similarity([text_embedding], [category_embedding])[0][0]
            if similarity >= threshold:
                predictions.append((category, similarity))
                
        # 유사도 순으로 정렬
        predictions.sort(key=lambda x: x[1], reverse=True)
        return [pred[0] for pred in predictions]
        
    def predict_v2(self, text: str, threshold: float = 0.5) -> List[str]:
        """V2: NER + 키워드 평균 임베딩"""
        # NER 전처리
        processed_text = self.preprocess_with_ner(text)
        text_embedding = self.sentence_model.encode([processed_text])[0]
        
        predictions = []
        for category, category_embedding in self.category_embeddings_v2.items():
            similarity = cosine_similarity([text_embedding], [category_embedding])[0][0]
            if similarity >= threshold:
                predictions.append((category, similarity))
                
        # 유사도 순으로 정렬
        predictions.sort(key=lambda x: x[1], reverse=True)
        return [pred[0] for pred in predictions]


In [ ]:
# PerformanceTester 클래스에 벤치마크 메서드 추가
def benchmark(self, test_data: List[Tuple], num_runs: int = 3):
    """성능 벤치마크"""
    results = {
        'v1_times': [],
        'v2_times': [],
        'v1_accuracy': [],
        'v2_accuracy': []
    }
    
    for run in range(num_runs):
        print(f"\\nRun {run + 1}/{num_runs}")
        
        # V1 테스트
        start_time = time.time()
        v1_correct = 0
        
        for text, true_categories in tqdm(test_data, desc="V1 Testing"):
            predictions = self.predict_v1(text)
            if any(pred in true_categories for pred in predictions):
                v1_correct += 1
                
        v1_time = time.time() - start_time
        v1_accuracy = v1_correct / len(test_data)
        
        results['v1_times'].append(v1_time)
        results['v1_accuracy'].append(v1_accuracy)
        
        # V2 테스트
        start_time = time.time()
        v2_correct = 0
        
        for text, true_categories in tqdm(test_data, desc="V2 Testing"):
            predictions = self.predict_v2(text)
            if any(pred in true_categories for pred in predictions):
                v2_correct += 1
                
        v2_time = time.time() - start_time
        v2_accuracy = v2_correct / len(test_data)
        
        results['v2_times'].append(v2_time)
        results['v2_accuracy'].append(v2_accuracy)
        
        print(f"V1 - Time: {v1_time:.2f}s, Accuracy: {v1_accuracy:.3f}")
        print(f"V2 - Time: {v2_time:.2f}s, Accuracy: {v2_accuracy:.3f}")
        
    return results

# 메서드를 클래스에 바인딩
PerformanceTester.benchmark = benchmark


In [ ]:
# 각 디바이스별 성능 테스트
devices_to_test = ['cpu']

if torch.cuda.is_available():
    devices_to_test.append('cuda')
    
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    devices_to_test.append('mps')

print(f"Testing on devices: {devices_to_test}")

all_results = {}

for device in devices_to_test:
    print(f"\\n{'='*50}")
    print(f"Testing on {device.upper()}")
    print(f"{'='*50}")
    
    try:
        tester = PerformanceTester(device=device)
        results = tester.benchmark(test_data, num_runs=2)  # Colab에서는 2번만
        all_results[device] = results
        
        # 평균 결과 출력
        print(f"\\n{device.upper()} Average Results:")
        print(f"V1 - Avg Time: {np.mean(results['v1_times']):.2f}s ± {np.std(results['v1_times']):.2f}")
        print(f"V1 - Avg Accuracy: {np.mean(results['v1_accuracy']):.3f} ± {np.std(results['v1_accuracy']):.3f}")
        print(f"V2 - Avg Time: {np.mean(results['v2_times']):.2f}s ± {np.std(results['v2_times']):.2f}")
        print(f"V2 - Avg Accuracy: {np.mean(results['v2_accuracy']):.3f} ± {np.std(results['v2_accuracy']):.3f}")
        
    except Exception as e:
        print(f"Error testing on {device}: {e}")
        continue


In [ ]:
# 결과 비교 및 시각화
if len(all_results) > 1:
    # 성능 비교 차트
    devices = list(all_results.keys())
    v1_times = [np.mean(all_results[device]['v1_times']) for device in devices]
    v2_times = [np.mean(all_results[device]['v2_times']) for device in devices]
    v1_accuracies = [np.mean(all_results[device]['v1_accuracy']) for device in devices]
    v2_accuracies = [np.mean(all_results[device]['v2_accuracy']) for device in devices]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 시간 비교
    x = np.arange(len(devices))
    width = 0.35
    
    ax1.bar(x - width/2, v1_times, width, label='V1 (Simple)', alpha=0.8, color='skyblue')
    ax1.bar(x + width/2, v2_times, width, label='V2 (NER+Keywords)', alpha=0.8, color='lightcoral')
    ax1.set_xlabel('Device')
    ax1.set_ylabel('Time (seconds)')
    ax1.set_title('Inference Time Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels([d.upper() for d in devices])
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 정확도 비교
    ax2.bar(x - width/2, v1_accuracies, width, label='V1 (Simple)', alpha=0.8, color='skyblue')
    ax2.bar(x + width/2, v2_accuracies, width, label='V2 (NER+Keywords)', alpha=0.8, color='lightcoral')
    ax2.set_xlabel('Device')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy Comparison')
    ax2.set_xticks(x)
    ax2.set_xticklabels([d.upper() for d in devices])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    # 성능 향상률 계산
    if 'cpu' in all_results:
        cpu_v1_time = np.mean(all_results['cpu']['v1_times'])
        cpu_v2_time = np.mean(all_results['cpu']['v2_times'])
        
        print("\\n" + "="*50)
        print("Performance Improvement vs CPU")
        print("="*50)
        
        for device in devices:
            if device != 'cpu':
                device_v1_time = np.mean(all_results[device]['v1_times'])
                device_v2_time = np.mean(all_results[device]['v2_times'])
                
                v1_speedup = cpu_v1_time / device_v1_time
                v2_speedup = cpu_v2_time / device_v2_time
                
                print(f"{device.upper()} vs CPU:")
                print(f"  V1 Speedup: {v1_speedup:.2f}x")
                print(f"  V2 Speedup: {v2_speedup:.2f}x")
                print()
else:
    print("Only one device tested, skipping comparison charts.")


In [ ]:
# 샘플 예측 결과 확인
if all_results:
    # 가장 좋은 성능의 디바이스 사용
    best_device = list(all_results.keys())[0]
    if 'cuda' in all_results:
        best_device = 'cuda'
    elif 'mps' in all_results:
        best_device = 'mps'
        
    tester = PerformanceTester(device=best_device)
    
    print(f"\\nSample predictions using {best_device.upper()}:")
    print("="*60)
    
    for i, (text, true_categories) in enumerate(test_data[:5]):
        v1_pred = tester.predict_v1(text)
        v2_pred = tester.predict_v2(text)
        
        print(f"\\nSample {i+1}: {text}")
        print(f"True categories: {true_categories}")
        print(f"V1 predictions: {v1_pred}")
        print(f"V2 predictions: {v2_pred}")
        
        # 정확도 체크
        v1_correct = any(pred in true_categories for pred in v1_pred)
        v2_correct = any(pred in true_categories for pred in v2_pred)
        
        print(f"V1 correct: {'✓' if v1_correct else '✗'}, V2 correct: {'✓' if v2_correct else '✗'}")


In [ ]:
## 결론 및 분석

### MPS vs CUDA vs CPU 성능 분석:

#### **왜 MPS가 기대만큼 빠르지 않을까?**

1. **작은 배치 사이즈**: 
   - 텍스트 임베딩은 보통 한 번에 하나씩 처리
   - GPU는 큰 배치에서 진가를 발휘하는데, 작은 배치에서는 오버헤드가 큼

2. **메모리 전송 오버헤드**:
   - CPU → GPU 데이터 전송 시간
   - GPU → CPU 결과 전송 시간
   - 실제 연산 시간보다 전송 시간이 더 클 수 있음

3. **모델 크기**:
   - SentenceTransformer는 상대적으로 작은 모델
   - GPU의 병렬 처리 능력을 충분히 활용하지 못함

4. **최적화 부족**:
   - 모델이 GPU 최적화가 덜 되어 있을 수 있음
   - 메모리 할당 패턴이 GPU에 최적화되지 않았을 수 있음

#### **언제 GPU가 유리할까?**

- **배치 크기가 큰 경우** (100개 이상의 텍스트를 한 번에 처리)
- **더 큰 모델** (BERT-large, GPT 등)
- **긴 텍스트** (512 토큰 이상)
- **실시간이 아닌 배치 처리**

#### **실제 운영 환경에서의 권장사항**

1. **개발/테스트**: CPU 사용 (간단하고 빠름)
2. **대량 배치 처리**: GPU 사용 (배치 크기 늘려서)
3. **실시간 API**: CPU 또는 작은 GPU (지연시간 중요)
